In [1]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

import  load_dotenv
load_dotenv.load_dotenv("../../.env")

True

몇가지 컨셉으로 RAG를 다룬다고 한다  
1. Loading  
Data ingestion: 다양한 데이터 소스로부터 데이터를 얻어냄
2. Indexing
Make Vector embedding
3. Storing
re-index 없이 바로 저장하기
4. Querying
서브쿼리, 멀티쿼리, Hybrid 쿼리 등 다양한 패턴으로 질문
5. Evaluation
query가 얼마나 정확히, 빠르게 대답이 생성되었는지  

그 중에 llamaindex는 이 세가지를 중점적으로 다룬다함
### 1. Loading
### 2. Indexing
### 3. Querying

# Loading stage
- Document: Data, API output, 어떤 DB로든부터 얻어진 데이터를 보관하는 컨테이너  
- Node: 청크를 나타내는 단위, 메타데이터에 추가로 **원본 문서와의 연관을 보관함**->다른 노드연계가능
- Reader: 추상적개념 data connector의 구현체, Ingestion을 담당하는듯

In [2]:
from llama_index.core import Document, VectorStoreIndex

text_list = ["Hello", "World"]
documents = [Document(text=t) for t in text_list]

index = VectorStoreIndex.from_documents(documents)

In [5]:
from llama_index.core.node_parser import SentenceSplitter

# load documents
...

# parse nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# build index
index = VectorStoreIndex(nodes)

In [7]:
nodes

[TextNode(id_='98184264-d9e6-4396-a2ff-ce101a64e5ff', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c34372e4-438d-44e7-8a36-5a4b787b63ad', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='52c87cd40ccfbd7873af4180fced6d38803d4c3684ed60f6513e8d16077e5b8e')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Hello', mimetype='text/plain', start_char_idx=0, end_char_idx=5, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='c4755e65-e061-479f-b835-43d91a195521', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ccac99ff-d8b1-4f97-879a-0d31b9e8c21f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ff5897aa373b6f1f671166b31e806183334dbfd64eb05f8970ca9555a8643e6e')}, metadata_template='{key}: {va

In [13]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()

## Inject metadata

In [14]:
document = Document(
    text="text",
    metadata={"filename": "<doc_file_name>", "category": "<category>"},
)

In [15]:
document.metadata = {"filename": "<doc_file_name>"}

In [ ]:
from llama_index.core import SimpleDirectoryReader

filename_fn = lambda filename: {"file_name": filename}

# automatically sets the metadata of each document according to filename_fn
## 이게 뭔...?
documents = SimpleDirectoryReader(
    "./data", file_metadata=filename_fn
).load_data()

## ID 지정

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data", filename_as_id=True).load_data()
print([x.doc_id for x in documents])